In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import extract_faces_keep_in_mem
from search_videos_in_directory_keep_in_mem import search_videos

ImportError: Traceback (most recent call last):
  File "/home/debanik/PycharmProjects/Detecting-Deepfakes/venv/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/debanik/PycharmProjects/Detecting-Deepfakes/venv/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/debanik/PycharmProjects/Detecting-Deepfakes/venv/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.10.0: cannot open shared object file: No suc